In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 719.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 9.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
  

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

In [ ]:
def build_gen2_model(features, outcomes):
  # Create the model
  inputs = layers.Input(shape=(features, outcomes))
  x = keras.layers.LSTM(150, return_sequences=True)(inputs)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.LSTM(150, return_sequences=True)(x)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Bidirectional(layers.LSTM(150, return_sequences=True))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Attention()([x, x])
  outputs = keras.layers.Dense(1, activation='linear')(x)
  model = Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer='adam', loss="mse")
  return model

In [ ]:
def train():
  tickers_df = pd.read_csv(data_path +'/training_tickers.csv')
  count = 0
  total = len(tickers_df)
  for ticker in tickers_df.iloc[:, 0]:
    print("training: ", name, " ticker: ", ticker, " count:", count, "of:", total)
    count+=1
    try:
      tickerObj = yf.download(tickers = ticker, interval = "1d")
      df = pd.DataFrame(tickerObj)
      df = df.reset_index()
    
      if len(df) > 0:
        history = train_config_model_against_df(name, model_config, df)
        if not name in histories:
          histories[name] = []
        histories[name].append(history)
    except Exception as inst:
      #raise inst
      print("Error: ", inst)

In [29]:
## I THINK WE CAN TOSS THIS AS NOT USEFUL

import plotly.graph_objects as go

def generate_df(window):
  ticker = "AAPL"

  # Download the data for the given ticker
  tickerObj = yf.download(tickers = ticker, interval = "1d")

  # Convert the data to a pandas dataframe
  df = pd.DataFrame(tickerObj)

  # Reset the index
  df = df.reset_index()

  # Calculate the 7-day moving average
  mawindow = df['Close'].rolling(window=window).mean()

  # Calculate the window-day standard deviation
  sdwindow = df['Close'].rolling(window=window).std()

  # Calculate the pivot points
  h2 = mawindow + 2 * sdwindow
  h1 = mawindow + sdwindow
  l1 = mawindow - sdwindow
  l2 = mawindow - 2 * sdwindow

  # Add the pivot points to the dataframe
  df['H2'] = h2
  df['H1'] = h1
  df['L1'] = l1
  df['L2'] = l2

  # Calculate the probability of reaching each pivot point
  df['PH2'] = df['High'].ge(h2).rolling(window=window).sum() / window
  df['PH1'] = df['High'].between(h1, h2 - 0.01).rolling(window=window).sum() / window
  df['PL1'] = df['Low'].between(l2, l1 - 0.01).rolling(window=window).sum() / window
  df['PL2'] = df['Low'].le(l2).rolling(window=window).sum() / window

  # Determine the color of each candlestick based on the highest probability
  df['color'] = np.where(df['PH2'] == df[['PH2', 'PH1', 'PL1', 'PL2']].max(axis=1), 'green',
                        np.where(df['PH1'] == df[['PH2', 'PH1', 'PL1', 'PL2']].max(axis=1), 'blue',
                                  np.where(df['PL1'] == df[['PH2', 'PH1', 'PL1', 'PL2']].max(axis=1), 'orange', 'red')))
  return df


df = generate_df(90).tail(365)

# Create the candlestick chart
fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                                     open=df['Open'],
                                     high=df['High'],
                                     low=df['Low'],
                                     close=df['Close'],                            
                                     showlegend=False)])

fig.add_trace(go.Scatter(x=df['Date'],
                         y=df['High'],
                         mode='markers',
                         marker=dict(color=df["color"], size=5),
                         name='PH2'))

fig.add_trace(go.Scatter(x=df['Date'], y=df['H2'], mode='lines', name='H2'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['H1'], mode='lines', name='H1'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['L1'], mode='lines', name='L1'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['L2'], mode='lines', name='L2'))

# Set the chart title and axis labels
fig.update_layout(title=ticker,
                  xaxis_title='Date',
                  yaxis_title='Price')

# Show the chart
fig.show()

[*********************100%***********************]  1 of 1 completed


In [ ]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,H2,H1,L1,L2,PH2,PH1,PL1,PL2,color
10275,2021-09-14,150.350006,151.070007,146.910004,148.119995,146.826492,109296300,155.696839,152.637753,146.519581,143.460495,0.366667,0.266667,0.033333,0.000000,green
10276,2021-09-15,148.559998,149.440002,146.369995,149.029999,147.728561,83281300,155.699130,152.666732,146.601935,143.569536,0.366667,0.233333,0.066667,0.000000,green
10277,2021-09-16,148.440002,148.970001,147.220001,148.789993,147.490646,68034100,155.684877,152.690272,146.701061,143.706456,0.366667,0.233333,0.066667,0.000000,green
10278,2021-09-17,148.820007,148.820007,145.759995,146.059998,144.784500,129868800,155.722949,152.692641,146.632025,143.601718,0.366667,0.233333,0.100000,0.000000,green
10279,2021-09-20,143.800003,144.839996,141.270004,142.940002,141.691757,123478900,155.974838,152.765252,146.346081,143.136495,0.366667,0.233333,0.100000,0.033333,green
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10635,2023-02-17,152.350006,153.000000,150.850006,152.550003,152.550003,59095900,161.085042,152.520854,135.392478,126.828290,0.200000,0.466667,0.066667,0.000000,blue
10636,2023-02-21,150.199997,151.300003,148.410004,148.479996,148.479996,58867200,160.901510,152.743421,136.427244,128.269155,0.200000,0.466667,0.033333,0.000000,blue
10637,2023-02-22,148.869995,149.949997,147.160004,148.910004,148.910004,51011300,160.652081,152.931374,137.489959,129.769251,0.200000,0.466667,0.033333,0.000000,blue
10638,2023-02-23,150.089996,150.339996,147.240005,149.399994,149.399994,48394200,160.335851,153.084425,138.581574,131.330148,0.200000,0.466667,0.000000,0.000000,blue
